In [1]:
# primary author michael

In [ ]:
# import standard libraries
from autograd import numpy as np
from autograd import grad
from autograd.misc.optimizers import adam, sgd
from autograd import scipy as sp
import autograd.numpy.random as npr
import pandas as pd
import numpy
import matplotlib.pyplot as plt
import sys
import time

# import our libraries
import bayes_helpers as bh
from utils import generate_data, run_toy_nn
from feed_forward import Feedforward
from nlm import NLM
from luna import LUNA

from config import *

### Generate Cubic Dataset

Generates 100 datapoints for train and 100 points for test according to the function

$$y = x^3 + \epsilon$$

$$\epsilon \sim N(0, 3^2)$$

In [ ]:
x_train, y_train, x_test = generate_data(training_seed)
#opt_params["max_iteration"] = max_iterations

### Train NLM

In [ ]:
t0 = time.time()
nlm = NLM(prior_variance,y_noise_variance, regularization_param_nlm, nlm_architecture, random_seed)
nlm.train(x_train,y_train, opt_params)
print(f"time: {np.round(time.time() - t0, 3)} seconds")

### Train LUNA

In [ ]:
t0 = time.time()
luna = LUNA(prior_variance, y_noise_variance, regularization_param_luna, similarity_param, luna_architecture, random_seed)
luna.train(x_train, y_train, opt_params)
print(f"time: {np.round(time.time() - t0, 3)} seconds")

### Compare Results
- notice they have identical post processing function calls

In [ ]:
x_valid, y_valid, x_test_null = generate_data(valid_seed)


In [ ]:
# LUNA took 
i = 0

fig,ax = plt.subplots(1,2 ,figsize=(10,5))
i = 0
for name, model in {"NLM":nlm,"LUNA":luna}.items():
    predictives, predictive_samples = model.predict(x_test)
    log_l = model.get_log_l(x_train,y_train,x_valid,y_valid)
    name = name + f"\n Log Likelihood: {round(log_l,2)}"
    ax[i]=bh.viz_pp_samples(x_train, y_train,x_test.flatten(),predictive_samples,name, ax[i])
    
    i +=1
plt.legend()
fig.savefig("figs/LUNA_NLM_Base.png")

### LUNA Over Regularized Params

In [ ]:
reg_params = [0.01,0.1,1,10]
times = []
lunas = []
opt_params["max_iteration"] = 3500
for reg_param in reg_params:
    print(reg_param)
    t0 = time.time()
    lunai = LUNA(prior_variance, y_noise_variance, regularization_param_luna, similarity_param ,luna_architecture, random_seed)

    #lunai = NLM(prior_variance, y_noise_variance, reg_param, nlm_architecture, random_seed)
    lunai.train(x_train, y_train, opt_params)
    lunas.append(lunai)
    times.append(np.round(time.time() - t0, 3))

In [ ]:
fig,ax = plt.subplots(2,2 ,figsize=(15,15))
i = 0
j= 0
k = 0
for model in lunas:
    predictives, predictive_samples = model.predict(x_test)
    log_l = model.get_log_l(x_train,y_train,x_valid,y_valid)
    name = f"LUNA: Reg = {reg_params[k]}" + f"\n Log Likelihood: {round(log_l,2)}" + f"\n Training Time: {round(times[k]/(60*60),2)} Hours"
    bh.viz_pp_samples(x_train, y_train,x_test.flatten(),predictive_samples,name, ax[i][j])
    
    j +=1
    if j == 2:
        i+=1
        j =0
    k +=1
plt.legend()
#plt.title ("Tuning Regularization with 10,000 Max Iterations")
fig.savefig("figs/LUNA_reg_tune3500.png")